# Maze making using graphs

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
November 2024   
December 2025

----

## Introduction

This document ([notebook]()) describes three ways of making mazes (or labyrinths) using graphs. The first two a based on rectangular grids, the third on a hexagonal grid.

All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph), [AAp1]. 
The package is ["Math::Nearest"](https://raku.land/zef:antononcube/Math::Nearest), [AAp2], for the construction of the third, hexagonal graph.

### TL;DR

Just see the maze pictures below. (And try to solve the mazes.)

### Procedure outline

The first maze is done by simple procedure which is actually some sort of cheating:

- A regular rectangular grid graph is generated with random weights associated with its edges.
- The (minimum) spanning tree for that graph is found.
- That tree is plotted with exaggeratedly big vertices and edges, so graph plot looks like a maze
    - This is "the cheat" -- the maze walls are not given by the graph.

The second maze is done "properly": 

- Two interlacing regular rectangular grid graphs are made.
- The second one has rows and columns that one less than rows and columns of the first one.
- The vertex coordinates of the second graph are at the centers of the rectangles of the first graph.
- The first graph provides the maze walls; the second graph is used to make paths through the maze.
    - Or, in other words, make a solvable maze. 
- Again, random weights are assigned to edges of the second graph and minimum spanning tree is found.
- There is a convenient formula that allows using the spanning tree edges to remove edges from the first graph.
- In that way a proper maze is derived. 

The third maze is again "properly" done using the procedure above with tow modifications:

- Two interlacing regular grid graphs are created: one is over a hexagonal grid, the other over a triangular grid.
    - The hexagonal grid graph provides the maze walls; the triangular grid graph the maze paths.
- Since the formula for wall removing is hard to derive a more robust and universal method used based on nearest neighbors.

------

## Setup

### Packages

Here are loaded the packages used in the rest of notebook:

In [44]:
use Graph;
use Graph::Classes;

use Math::DistanceFunctions;
use Math::Nearest;

use Data::TypeSystem;
use Data::Translators;

### Conversion

This sub is used to invoke the Graphviz graph layout engines:

In [45]:
sub dot-svg($input, Str:D :$engine = 'dot', Str:D :$format = 'svg') {
    my $temp-file = $*TMPDIR.child("temp-graph.dot");
    $temp-file.spurt: $input;
    my $svg-output = run($engine, $temp-file, "-T$format", :out).out.slurp-rest;
    unlink $temp-file;
    return $svg-output;
}

&dot-svg

-----

## Simple maze

In [47]:
my ($n, $m) = (10, 25);
my $g3 = Graph::Grid.new($n, $m, :!directed);
my $g4 = Graph.new($g3.edges(:dataset).map({ $_<weight> = random-real([10,1000]); $_}))

Graph(vertexes => 250, edges => 465, directed => False)

In [48]:
my $g5 = $g4.find-spanning-tree

Graph(vertexes => 250, edges => 249, directed => False)

Shortest path from first vertex (bottom-left) to last vertex (top-right):

In [49]:
my @path = $g5.find-shortest-path('0_0', "{$n-1}_{$m-1}");
@path.elems

44

Graph plot:

In [50]:
#% html
my $g6 = Graph.new($g5.edges);
$g6.vertex-coordinates = $g3.vertex-coordinates;
my %opts =
    background => "Black",
    :!node-labels,
    :!edge-lables,
    node-shape => 'square', 
    node-width => 0.7, 
    node-color => 'SteelBlue',
    node-fill-color => 'SteelBlue',
    edge-thickness => 52, 
    edge-color => 'SteelBlue',
    size => '10,6!',
    engine => 'neato';
$g6.dot(|%opts):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 9_18 -->
 
 9_18 
 
 
<!-- 8_4 -->
 
 8_4 
 
 
<!-- 9_4 -->
 
 9_4 
 
 
<!-- 8_4--9_4 -->
 
 8_4--9_4 
 
 
<!-- 0_19 -->
 
 0_19 
 
 
<!-- 1_19 -->
 
 1_19 
 
 
<!-- 0_19--1_19 -->
 
 0_19--1_19 
 
 
<!-- 0_20 -->
 
 0_20 
 
 
<!-- 0_19--0_20 -->
 
 0_19--0_20 
 
 
<!-- 3_7 -->
 
 3_7 
 
 
<!-- 3_8 -->
 
 3_8 
 
 
<!-- 3_7--3_8 -->
 
 3_7--3_8 
 
 
<!-- 2_10 -->
 
 2_10 
 
 
<!-- 3_10 -->
 
 3_10 
 
 
<!-- 2_10--3_10 -->
 
 2_10--3_10 
 
 
<!-- 2_9 -->
 
 2_9 
 
 
<!-- 2_10--2_9 -->
 
 2_10--2_9 
 
 
<!-- 3_22 -->
 
 3_22 
 
 
<!-- 7_17 -->
 
 7_17 
 
 
<!-- 7_18 -->
 
 7_18 
 
 
<!-- 7_17--7_18 -->
 
 7_17--7_18 
 
 
<!-- 8_17 -->
 
 8_17 
 
 
<!-- 7_17--8_17 -->
 
 7_17--8_17 
 
 
<!-- 0_22 -->
 
 0_22 
 
 
<!-- 0_23 -->
 
 0_23 
 
 
<!-- 0_22--0_23 -->
 
 0_22--0_23 
 
 
<!-- 7_21 -->
 
 7_21 
 
 
<!-- 8_21 -->
 
 8_21 
 
 
<!-- 7_21--8_21 -->
 
 7_21--8_21 
 
 
<!-- 7_22 -->
 
 7_22 
 
 
<!-- 7_21--7_22 -->
 
 7_21--7_22 
 
 
<!-- 8_23 -->
 
 8_23 
 
 
<!-- 8_24 -->
 
 8_24 
 
 
<!-- 8_23--8_24 -->
 
 8_23--8_24 
 
 
<!-- 9_12 -->
 
 9_12 
 
 
<!-- 9_13 -->
 
 9_13 
 
 
<!-- 9_12--9_13 -->
 
 9_12--9_13 
 
 
<!-- 2_7 -->
 
 2_7 
 
 
<!-- 2_8 -->
 
 2_8 
 
 
<!-- 2_7--2_8 -->
 
 2_7--2_8 
 
 
<!-- 8_9 -->
 
 8_9 
 
 
<!-- 7_23 -->
 
 7_23 
 
 
<!-- 1_9 -->
 
 1_9 
 
 
<!-- 3_5 -->
 
 3_5 
 
 
<!-- 4_18 -->
 
 4_18 
 
 
<!-- 0_12 -->
 
 0_12 
 
 
<!-- 0_13 -->
 
 0_13 
 
 
<!-- 0_12--0_13 -->
 
 0_12--0_13 
 
 
<!-- 1_21 -->
 
 1_21 
 
 
<!-- 4_11 -->
 
 4_11 
 
 
<!-- 3_23 -->
 
 3_23 
 
 
<!-- 4_7 -->
 
 4_7 
 
 
<!-- 8_7 -->
 
 8_7 
 
 
<!-- 8_8 -->
 
 8_8 
 
 
<!-- 8_7--8_8 -->
 
 8_7--8_8 
 
 
<!-- 6_16 -->
 
 6_16 
 
 
<!-- 6_17 -->
 
 6_17 
 
 
<!-- 6_16--6_17 -->
 
 6_16--6_17 
 
 
<!-- 9_8 -->
 
 9_8 
 
 
<!-- 8_20 -->
 
 8_20 
 
 
<!-- 6_12 -->
 
 6_12 
 
 
<!-- 6_13 -->
 
 6_13 
 
 
<!-- 6_12--6_13 -->
 
 6_12--6_13 
 
 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 1_2 -->
 
 1_2 
 
 
<!-- 0_2--1_2 -->
 
 0_2--1_2 
 
 
<!-- 6_7 -->
 
 6_7 
 
 
<!-- 7_13 -->
 
 7_13 
 
 
<!-- 6_13--7_13 -->
 
 6_13--7_13 
 
 
<!-- 5_19 -->
 
 5_19 
 
 
<!-- 8_1 -->
 
 8_1 
 
 
<!-- 8_2 -->
 
 8_2 
 
 
<!-- 8_1--8_2 -->
 
 8_1--8_2 
 
 
<!-- 9_1 -->
 
 9_1 
 
 
<!-- 8_1--9_1 -->
 
 8_1--9_1 
 
 
<!-- 6_18 -->
 
 6_18 
 
 
<!-- 6_18--7_18 -->
 
 6_18--7_18 
 
 
<!-- 5_3 -->
 
 5_3 
 
 
<!-- 6_3 -->
 
 6_3 
 
 
<!-- 5_3--6_3 -->
 
 5_3--6_3 
 
 
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 8_13 -->
 
 8_13 
 
 
<!-- 8_14 -->
 
 8_14 
 
 
<!-- 8_13--8_14 -->
 
 8_13--8_14 
 
 
<!-- 5_8 -->
 
 5_8 
 
 
<!-- 5_9 -->
 
 5_9 
 
 
<!-- 5_8--5_9 -->
 
 5_8--5_9 
 
 
<!-- 6_8 -->
 
 6_8 
 
 
<!-- 5_8--6_8 -->
 
 5_8--6_8 
 
 
<!-- 2_12 -->
 
 2_12 
 
 
<!-- 6_22 -->
 
 6_22 
 
 
<!-- 6_22--7_22 -->
 
 6_22--7_22 
 
 
<!-- 4_4 -->
 
 4_4 
 
 
<!-- 5_4 -->
 
 5_4 
 
 
<!-- 4_4--5_4 -->
 
 4_4--5_4 
 
 
<!-- 2_6 -->
 
 2_6 
 
 
<!-- 3_6 -->
 
 3_6 
 
 
<!-- 2_6--3_6 -->
 
 2_6--3_6 
 
 
<!-- 8_18 -->
 
 8_18 
 
 
<!-- 7_18--8_18 -->
 
 7_18--8_18 
 
 
<!-- 1_22 -->
 
 1_22 
 
 
<!-- 2_22 -->
 
 2_22 
 
 
<!-- 1_22--2_22 -->
 
 1_22--2_22 
 
 
<!-- 9_9 -->
 
 9_9 
 
 
<!-- 2_13 -->
 
 2_13 
 
 
<!-- 3_13 -->
 
 3_13 
 
 
<!-- 2_13--3_13 -->
 
 2_13--3_13 
 
 
<!-- 2_5 -->
 
 2_5 
 
 
<!-- 2_5--2_6 -->
 
 2_5--2_6 
 
 
<!-- 8_6 -->
 
 8_6 
 
 
<!-- 9_6 -->
 
 9_6 
 
 
<!-- 8_6--9_6 -->
 
 8_6--9_6 
 
 
<!-- 4_23 -->
 
 4_23 
 
 
<!-- 9_14 -->
 
 9_14 
 
 
<!-- 9_15 -->
 
 9_15 
 
 
<!-- 9_14--9_15 -->
 
 9_14--9_15 
 
 
<!-- 1_20 -->
 
 1_20 
 
 
<!-- 1_20--1_21 -->
 
 1_20--1_21 
 
 
<!-- 1_0 -->
 
 1_0 
 
 
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 1_0--2_0 -->
 
 1_0--2_0 
 
 
<!-- 3_2 -->
 
 3_2 
 
 
<!-- 9_21 -->
 
 9_21 
 
 
<!-- 9_22 -->
 
 9_22 
 
 
<!-- 9_21--9_22 -->
 
 9_21--9_22 
 
 
<!-- 6_20 -->
 
 6_20 
 
 
<!-- 5_16 -->
 
 5_16 
 

The "maze" above looks like a maze because the vertexes and edges are rectangular with matching sizes, and they are thicker than the spaces between them. In other words we are cheating. 

To make that cheating construction more clear, let us plot the shortest path from bottom left to top right and color the edges in pink (salmon) and the vertexes in red:

In [51]:
#%html
my $gPath = Graph::Path.new(@path);
$g6.dot(highlight => {Salmon => $gPath.edge-list, Red => $gPath.vertex-list}, |%opts):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 9_18 -->
 
 9_18 
 
 
<!-- 8_4 -->
 
 8_4 
 
 
<!-- 9_4 -->
 
 9_4 
 
 
<!-- 8_4--9_4 -->
 
 8_4--9_4 
 
 
<!-- 0_19 -->
 
 0_19 
 
 
<!-- 1_19 -->
 
 1_19 
 
 
<!-- 0_19--1_19 -->
 
 0_19--1_19 
 
 
<!-- 0_20 -->
 
 0_20 
 
 
<!-- 0_19--0_20 -->
 
 0_19--0_20 
 
 
<!-- 3_7 -->
 
 3_7 
 
 
<!-- 3_8 -->
 
 3_8 
 
 
<!-- 3_7--3_8 -->
 
 3_7--3_8 
 
 
<!-- 2_10 -->
 
 2_10 
 
 
<!-- 3_10 -->
 
 3_10 
 
 
<!-- 2_10--3_10 -->
 
 2_10--3_10 
 
 
<!-- 2_9 -->
 
 2_9 
 
 
<!-- 2_10--2_9 -->
 
 2_10--2_9 
 
 
<!-- 3_22 -->
 
 3_22 
 
 
<!-- 7_17 -->
 
 7_17 
 
 
<!-- 7_18 -->
 
 7_18 
 
 
<!-- 7_17--7_18 -->
 
 7_17--7_18 
 
 
<!-- 8_17 -->
 
 8_17 
 
 
<!-- 7_17--8_17 -->
 
 7_17--8_17 
 
 
<!-- 0_22 -->
 
 0_22 
 
 
<!-- 0_23 -->
 
 0_23 
 
 
<!-- 0_22--0_23 -->
 
 0_22--0_23 
 
 
<!-- 7_21 -->
 
 7_21 
 
 
<!-- 8_21 -->
 
 8_21 
 
 
<!-- 7_21--8_21 -->
 
 7_21--8_21 
 
 
<!-- 7_22 -->
 
 7_22 
 
 
<!-- 7_21--7_22 -->
 
 7_21--7_22 
 
 
<!-- 8_23 -->
 
 8_23 
 
 
<!-- 8_24 -->
 
 8_24 
 
 
<!-- 8_23--8_24 -->
 
 8_23--8_24 
 
 
<!-- 9_12 -->
 
 9_12 
 
 
<!-- 9_13 -->
 
 9_13 
 
 
<!-- 9_12--9_13 -->
 
 9_12--9_13 
 
 
<!-- 2_7 -->
 
 2_7 
 
 
<!-- 2_8 -->
 
 2_8 
 
 
<!-- 2_7--2_8 -->
 
 2_7--2_8 
 
 
<!-- 8_9 -->
 
 8_9 
 
 
<!-- 7_23 -->
 
 7_23 
 
 
<!-- 1_9 -->
 
 1_9 
 
 
<!-- 3_5 -->
 
 3_5 
 
 
<!-- 4_18 -->
 
 4_18 
 
 
<!-- 0_12 -->
 
 0_12 
 
 
<!-- 0_13 -->
 
 0_13 
 
 
<!-- 0_12--0_13 -->
 
 0_12--0_13 
 
 
<!-- 1_21 -->
 
 1_21 
 
 
<!-- 4_11 -->
 
 4_11 
 
 
<!-- 3_23 -->
 
 3_23 
 
 
<!-- 4_7 -->
 
 4_7 
 
 
<!-- 8_7 -->
 
 8_7 
 
 
<!-- 8_8 -->
 
 8_8 
 
 
<!-- 8_7--8_8 -->
 
 8_7--8_8 
 
 
<!-- 6_16 -->
 
 6_16 
 
 
<!-- 6_17 -->
 
 6_17 
 
 
<!-- 6_16--6_17 -->
 
 6_16--6_17 
 
 
<!-- 9_8 -->
 
 9_8 
 
 
<!-- 8_20 -->
 
 8_20 
 
 
<!-- 6_12 -->
 
 6_12 
 
 
<!-- 6_13 -->
 
 6_13 
 
 
<!-- 6_12--6_13 -->
 
 6_12--6_13 
 
 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 1_2 -->
 
 1_2 
 
 
<!-- 0_2--1_2 -->
 
 0_2--1_2 
 
 
<!-- 6_7 -->
 
 6_7 
 
 
<!-- 7_13 -->
 
 7_13 
 
 
<!-- 6_13--7_13 -->
 
 6_13--7_13 
 
 
<!-- 5_19 -->
 
 5_19 
 
 
<!-- 8_1 -->
 
 8_1 
 
 
<!-- 8_2 -->
 
 8_2 
 
 
<!-- 8_1--8_2 -->
 
 8_1--8_2 
 
 
<!-- 9_1 -->
 
 9_1 
 
 
<!-- 8_1--9_1 -->
 
 8_1--9_1 
 
 
<!-- 6_18 -->
 
 6_18 
 
 
<!-- 6_18--7_18 -->
 
 6_18--7_18 
 
 
<!-- 5_3 -->
 
 5_3 
 
 
<!-- 6_3 -->
 
 6_3 
 
 
<!-- 5_3--6_3 -->
 
 5_3--6_3 
 
 
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 8_13 -->
 
 8_13 
 
 
<!-- 8_14 -->
 
 8_14 
 
 
<!-- 8_13--8_14 -->
 
 8_13--8_14 
 
 
<!-- 5_8 -->
 
 5_8 
 
 
<!-- 5_9 -->
 
 5_9 
 
 
<!-- 5_8--5_9 -->
 
 5_8--5_9 
 
 
<!-- 6_8 -->
 
 6_8 
 
 
<!-- 5_8--6_8 -->
 
 5_8--6_8 
 
 
<!-- 2_12 -->
 
 2_12 
 
 
<!-- 6_22 -->
 
 6_22 
 
 
<!-- 6_22--7_22 -->
 
 6_22--7_22 
 
 
<!-- 4_4 -->
 
 4_4 
 
 
<!-- 5_4 -->
 
 5_4 
 
 
<!-- 4_4--5_4 -->
 
 4_4--5_4 
 
 
<!-- 2_6 -->
 
 2_6 
 
 
<!-- 3_6 -->
 
 3_6 
 
 
<!-- 2_6--3_6 -->
 
 2_6--3_6 
 
 
<!-- 8_18 -->
 
 8_18 
 
 
<!-- 7_18--8_18 -->
 
 7_18--8_18 
 
 
<!-- 1_22 -->
 
 1_22 
 
 
<!-- 2_22 -->
 
 2_22 
 
 
<!-- 1_22--2_22 -->
 
 1_22--2_22 
 
 
<!-- 9_9 -->
 
 9_9 
 
 
<!-- 2_13 -->
 
 2_13 
 
 
<!-- 3_13 -->
 
 3_13 
 
 
<!-- 2_13--3_13 -->
 
 2_13--3_13 
 
 
<!-- 2_5 -->
 
 2_5 
 
 
<!-- 2_5--2_6 -->
 
 2_5--2_6 
 
 
<!-- 8_6 -->
 
 8_6 
 
 
<!-- 9_6 -->
 
 9_6 
 
 
<!-- 8_6--9_6 -->
 
 8_6--9_6 
 
 
<!-- 4_23 -->
 
 4_23 
 
 
<!-- 9_14 -->
 
 9_14 
 
 
<!-- 9_15 -->
 
 9_15 
 
 
<!-- 9_14--9_15 -->
 
 9_14--9_15 
 
 
<!-- 1_20 -->
 
 1_20 
 
 
<!-- 1_20--1_21 -->
 
 1_20--1_21 
 
 
<!-- 1_0 -->
 
 1_0 
 
 
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 1_0--2_0 -->
 
 1_0--2_0 
 
 
<!-- 3_2 -->
 
 3_2 
 
 
<!-- 9_21 -->
 
 9_21 
 
 
<!-- 9_22 -->
 
 9_22 
 
 
<!-- 9_21--9_22 -->
 
 9_21--9_22 
 
 
<!-- 6_20 -->
 
 6_20 
 
 
<!-- 5_16 -->
 
 5_16 
 

-----

## Proper maze

A ***proper maze*** is a maze given with its walls. (Not with the space between walls.) 

**Remark:** For didactical reasons, the maze in this section is small so the steps can be easily followed.

Make a two regular graphs, one for the maze walls, the other for the maze paths:

In [ ]:
# Maze dimensions
my ($n, $m) = (6, 12) »*» 1;

# Walls graph
my $g1 = Graph::Grid.new($n, $m, prefix => 'w');

# Space graph
my $g2 = Graph::Grid.new($n-1, $m-1);
$g2.vertex-coordinates = $g2.vertex-coordinates.map({ $_.key => $_.value >>+>> 0.5 }).Hash;

$g2

Graph(vertexes => 55, edges => 94, directed => False)

Maze path graph:

In [53]:
my $mazePath = Graph.new($g2.edges(:dataset).map({ $_<weight> = random-real([10,1000]); $_}));
$mazePath = $mazePath.find-spanning-tree;
$mazePath.vertex-coordinates = $g2.vertex-coordinates;

$mazePath

Graph(vertexes => 55, edges => 54, directed => False)

Combined graph:

In [54]:
my $g3 = Graph.new([|$g1.edges, |$mazePath.edges]);
$g3.vertex-coordinates = [|$g1.vertex-coordinates, |$mazePath.vertex-coordinates].Hash;

$g3

Graph(vertexes => 127, edges => 180, directed => False)

Plot the combined graph:

In [55]:
#% html
$g3.dot(
    highlight => $mazePath,
    :node-labels,
    background => "Black",
    node-width => 0.45,
    node-height => 0.2,
    edge-width => 4,
    size => '10,10!',
    engine => 'neato'
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w5_11 -->
 
 w5_11 
 
 w5_11 
 
<!-- 0_6 -->
 
 0_6 
 
 0_6 
 
<!-- 0_7 -->
 
 0_7 
 
 0_7 
 
<!-- 0_6--0_7 -->
 
 0_6--0_7 
 
 
<!-- 1_6 -->
 
 1_6 
 
 1_6 
 
<!-- 0_6--1_6 -->
 
 0_6--1_6 
 
 
<!-- w0_8 -->
 
 w0_8 
 
 w0_8 
 
<!-- w0_9 -->
 
 w0_9 
 
 w0_9 
 
<!-- w0_8--w0_9 -->
 
 w0_8--w0_9 
 
 
<!-- w1_8 -->
 
 w1_8 
 
 w1_8 
 
<!-- w0_8--w1_8 -->
 
 w0_8--w1_8 
 
 
<!-- 4_5 -->
 
 4_5 
 
 4_5 
 
<!-- 0_5 -->
 
 0_5 
 
 0_5 
 
<!-- 4_1 -->
 
 4_1 
 
 4_1 
 
<!-- 4_2 -->
 
 4_2 
 
 4_2 
 
<!-- 4_1--4_2 -->
 
 4_1--4_2 
 
 
<!-- w4_4 -->
 
 w4_4 
 
 w4_4 
 
<!-- w4_5 -->
 
 w4_5 
 
 w4_5 
 
<!-- w4_4--w4_5 -->
 
 w4_4--w4_5 
 
 
<!-- w5_4 -->
 
 w5_4 
 
 w5_4 
 
<!-- w4_4--w5_4 -->
 
 w4_4--w5_4 
 
 
<!-- w5_10 -->
 
 w5_10 
 
 w5_10 
 
<!-- w5_10--w5_11 -->
 
 w5_10--w5_11 
 
 
<!-- w5_9 -->
 
 w5_9 
 
 w5_9 
 
<!-- w5_10--w5_9 -->
 
 w5_10--w5_9 
 
 
<!-- w5_3 -->
 
 w5_3 
 
 w5_3 
 
<!-- w5_3--w5_4 -->
 
 w5_3--w5_4 
 
 
<!-- w3_11 -->
 
 w3_11 
 
 w3_11 
 
<!-- w4_11 -->
 
 w4_11 
 
 w4_11 
 
<!-- w3_11--w4_11 -->
 
 w3_11--w4_11 
 
 
<!-- 0_3 -->
 
 0_3 
 
 0_3 
 
<!-- 0_4 -->
 
 0_4 
 
 0_4 
 
<!-- 0_3--0_4 -->
 
 0_3--0_4 
 
 
<!-- 2_6 -->
 
 2_6 
 
 2_6 
 
<!-- 3_1 -->
 
 3_1 
 
 3_1 
 
<!-- 3_1--4_1 -->
 
 3_1--4_1 
 
 
<!-- w1_5 -->
 
 w1_5 
 
 w1_5 
 
<!-- w2_5 -->
 
 w2_5 
 
 w2_5 
 
<!-- w1_5--w2_5 -->
 
 w1_5--w2_5 
 
 
<!-- w1_6 -->
 
 w1_6 
 
 w1_6 
 
<!-- w1_5--w1_6 -->
 
 w1_5--w1_6 
 
 
<!-- 3_0 -->
 
 3_0 
 
 3_0 
 
<!-- 3_0--3_1 -->
 
 3_0--3_1 
 
 
<!-- 4_0 -->
 
 4_0 
 
 4_0 
 
<!-- 3_0--4_0 -->
 
 3_0--4_0 
 
 
<!-- 3_3 -->
 
 3_3 
 
 3_3 
 
<!-- 1_7 -->
 
 1_7 
 
 1_7 
 
<!-- 2_7 -->
 
 2_7 
 
 2_7 
 
<!-- 1_7--2_7 -->
 
 1_7--2_7 
 
 
<!-- 1_8 -->
 
 1_8 
 
 1_8 
 
<!-- 1_7--1_8 -->
 
 1_7--1_8 
 
 
<!-- w0_4 -->
 
 w0_4 
 
 w0_4 
 
<!-- w1_4 -->
 
 w1_4 
 
 w1_4 
 
<!-- w0_4--w1_4 -->
 
 w0_4--w1_4 
 
 
<!-- w0_5 -->
 
 w0_5 
 
 w0_5 
 
<!-- w0_4--w0_5 -->
 
 w0_4--w0_5 
 
 
<!-- 2_8 -->
 
 2_8 
 
 2_8 
 
<!-- 2_9 -->
 
 2_9 
 
 2_9 
 
<!-- 2_8--2_9 -->
 
 2_8--2_9 
 
 
<!-- 0_4--0_5 -->
 
 0_4--0_5 
 
 
<!-- 1_4 -->
 
 1_4 
 
 1_4 
 
<!-- 0_4--1_4 -->
 
 0_4--1_4 
 
 
<!-- w5_0 -->
 
 w5_0 
 
 w5_0 
 
<!-- w5_1 -->
 
 w5_1 
 
 w5_1 
 
<!-- w5_0--w5_1 -->
 
 w5_0--w5_1 
 
 
<!-- 3_7 -->
 
 3_7 
 
 3_7 
 
<!-- 4_7 -->
 
 4_7 
 
 4_7 
 
<!-- 3_7--4_7 -->
 
 3_7--4_7 
 
 
<!-- w1_4--w1_5 -->
 
 w1_4--w1_5 
 
 
<!-- w2_4 -->
 
 w2_4 
 
 w2_4 
 
<!-- w1_4--w2_4 -->
 
 w1_4--w2_4 
 
 
<!-- w4_9 -->
 
 w4_9 
 
 w4_9 
 
<!-- w4_9--w5_9 -->
 
 w4_9--w5_9 
 
 
<!-- 1_2 -->
 
 1_2 
 
 1_2 
 
<!-- 2_2 -->
 
 2_2 
 
 2_2 
 
<!-- 1_2--2_2 -->
 
 1_2--2_2 
 
 
<!-- 1_3 -->
 
 1_3 
 
 1_3 
 
<!-- 1_2--1_3 -->
 
 1_2--1_3 
 
 
<!-- 3_4 -->
 
 3_4 
 
 3_4 
 
<!-- 3_5 -->
 
 3_5 
 
 3_5 
 
<!-- 3_4--3_5 -->
 
 3_4--3_5 
 
 
<!-- w0_0 -->
 
 w0_0 
 
 w0_0 
 
<!-- w0_1 -->
 
 w0_1 
 
 w0_1 
 
<!-- w0_0--w0_1 -->
 
 w0_0--w0_1 
 
 
<!-- w1_0 -->
 
 w1_0 
 
 w1_0 
 
<!-- w0_0--w1_0 -->
 
 w0_0--w1_0 
 
 
<!-- 4_9 -->
 
 4_9 
 
 4_9 
 
<!-- w2_0 -->
 
 w2_0 
 
 w2_0 
 
<!-- w3_0 -->
 
 w3_0 
 
 w3_0 
 
<!-- w2_0--w3_0 -->
 
 w2_0--w3_0 
 
 
<!-- w2_1 -->
 
 w2_1 
 
 w2_1 
 
<!-- w2_0--w2_1 -->
 
 w2_0--w2_1 
 
 
<!-- w4_10 -->
 
 w4_10 
 
 w4_10 
 
<!-- w4_10--w5_10 -->
 
 w4_10--w5_10 
 
 
<!-- w4_10--w4_9 -->
 
 w4_10--w4_9 
 
 
<!-- w4_10--w4_11 -->
 
 w4_10--w4_11 
 
 
<!-- w3_9 -->
 
 w3_9 
 
 w3_9 
 
<!-- w3_9--w4_9 -->
 
 w3_9--w4_9 
 
 
<!-- 3_8 -->
 
 3_8 
 
 3_8 
 
<!-- 3_9 -->
 
 3_9 
 
 3_9 
 
<!-- 3_8--3_9 -->
 
 3_8--3_9 
 
 
<!-- w3_5 -->
 
 w3_5 
 
 w3_5 
 
<!-- w2_5--w3_5 -->
 
 w2_5--w3_5 
 
 
<!-- w2_6 -->
 
 w2_6 
 
 w2_6 
 
<!-- w2_5--w2_6 -->
 
 w2_5--w2_6 
 
 
<!-- w1_9 -->
 
 w1_9 
 
 w1_9 
 
<!-- w2_9 -->
 
 w2_9 
 
 w2_9 
 
<!-- w1_9--w2_9 -->
 
 w1_9-

Remove wall edges using a formula:

In [56]:
my $g4 = $g3.clone;
for $mazePath.edges -> $e {

    my ($i, $j) = |$e.key.split('_')».Int;
    my ($i2, $j2) = |$e.value.split('_')».Int;
    
    if $i2 < $i || $j2 < $j { 
        ($i2, $j2, $i, $j) = ($i, $j, $i2, $j2)
    }

    # Horizontal
    if $i == $i2 && $j < $j2 {
        $g4 = $g4.edge-delete( "w{$i2}_{$j2}" => "w{$i2+1}_{$j2}")
    }

    # Vertical
    if $j == $j2 && $i < $i2 {
        $g4 = $g4.edge-delete( "w{$i2}_{$j2}" => "w{$i2}_{$j2+1}")
    }
}

$g4

Graph(vertexes => 127, edges => 126, directed => False)

Plot wall-graph and maze-space graph:

In [57]:
#% html
$g4.dot(
    highlight => $mazePath,
    :!node-labels,
    background => "Black",
    node-width => 0.25,
    node-height => 0.2,
    edge-width => 4,
    size => '10,10!',
    engine => 'neato'
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w5_11 -->
 
 w5_11 
 
 
<!-- 0_6 -->
 
 0_6 
 
 
<!-- 0_7 -->
 
 0_7 
 
 
<!-- 0_6--0_7 -->
 
 0_6--0_7 
 
 
<!-- 1_6 -->
 
 1_6 
 
 
<!-- 0_6--1_6 -->
 
 0_6--1_6 
 
 
<!-- w0_8 -->
 
 w0_8 
 
 
<!-- w0_9 -->
 
 w0_9 
 
 
<!-- w0_8--w0_9 -->
 
 w0_8--w0_9 
 
 
<!-- w1_8 -->
 
 w1_8 
 
 
<!-- w0_8--w1_8 -->
 
 w0_8--w1_8 
 
 
<!-- 4_5 -->
 
 4_5 
 
 
<!-- 0_5 -->
 
 0_5 
 
 
<!-- 4_1 -->
 
 4_1 
 
 
<!-- 4_2 -->
 
 4_2 
 
 
<!-- 4_1--4_2 -->
 
 4_1--4_2 
 
 
<!-- w4_4 -->
 
 w4_4 
 
 
<!-- w4_5 -->
 
 w4_5 
 
 
<!-- w4_4--w4_5 -->
 
 w4_4--w4_5 
 
 
<!-- w5_10 -->
 
 w5_10 
 
 
<!-- w5_10--w5_11 -->
 
 w5_10--w5_11 
 
 
<!-- w5_9 -->
 
 w5_9 
 
 
<!-- w5_10--w5_9 -->
 
 w5_10--w5_9 
 
 
<!-- w5_3 -->
 
 w5_3 
 
 
<!-- w5_4 -->
 
 w5_4 
 
 
<!-- w5_3--w5_4 -->
 
 w5_3--w5_4 
 
 
<!-- w3_11 -->
 
 w3_11 
 
 
<!-- w4_11 -->
 
 w4_11 
 
 
<!-- w3_11--w4_11 -->
 
 w3_11--w4_11 
 
 
<!-- 0_3 -->
 
 0_3 
 
 
<!-- 0_4 -->
 
 0_4 
 
 
<!-- 0_3--0_4 -->
 
 0_3--0_4 
 
 
<!-- 2_6 -->
 
 2_6 
 
 
<!-- 3_1 -->
 
 3_1 
 
 
<!-- 3_1--4_1 -->
 
 3_1--4_1 
 
 
<!-- w1_5 -->
 
 w1_5 
 
 
<!-- w2_5 -->
 
 w2_5 
 
 
<!-- w1_5--w2_5 -->
 
 w1_5--w2_5 
 
 
<!-- w1_6 -->
 
 w1_6 
 
 
<!-- w1_5--w1_6 -->
 
 w1_5--w1_6 
 
 
<!-- 3_0 -->
 
 3_0 
 
 
<!-- 3_0--3_1 -->
 
 3_0--3_1 
 
 
<!-- 4_0 -->
 
 4_0 
 
 
<!-- 3_0--4_0 -->
 
 3_0--4_0 
 
 
<!-- 3_3 -->
 
 3_3 
 
 
<!-- 1_7 -->
 
 1_7 
 
 
<!-- 2_7 -->
 
 2_7 
 
 
<!-- 1_7--2_7 -->
 
 1_7--2_7 
 
 
<!-- 1_8 -->
 
 1_8 
 
 
<!-- 1_7--1_8 -->
 
 1_7--1_8 
 
 
<!-- w0_4 -->
 
 w0_4 
 
 
<!-- w0_5 -->
 
 w0_5 
 
 
<!-- w0_4--w0_5 -->
 
 w0_4--w0_5 
 
 
<!-- 2_8 -->
 
 2_8 
 
 
<!-- 2_9 -->
 
 2_9 
 
 
<!-- 2_8--2_9 -->
 
 2_8--2_9 
 
 
<!-- 0_4--0_5 -->
 
 0_4--0_5 
 
 
<!-- 1_4 -->
 
 1_4 
 
 
<!-- 0_4--1_4 -->
 
 0_4--1_4 
 
 
<!-- w5_0 -->
 
 w5_0 
 
 
<!-- w5_1 -->
 
 w5_1 
 
 
<!-- w5_0--w5_1 -->
 
 w5_0--w5_1 
 
 
<!-- 3_7 -->
 
 3_7 
 
 
<!-- 4_7 -->
 
 4_7 
 
 
<!-- 3_7--4_7 -->
 
 3_7--4_7 
 
 
<!-- w1_4 -->
 
 w1_4 
 
 
<!-- w2_4 -->
 
 w2_4 
 
 
<!-- w1_4--w2_4 -->
 
 w1_4--w2_4 
 
 
<!-- w4_9 -->
 
 w4_9 
 
 
<!-- 1_2 -->
 
 1_2 
 
 
<!-- 2_2 -->
 
 2_2 
 
 
<!-- 1_2--2_2 -->
 
 1_2--2_2 
 
 
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 1_2--1_3 -->
 
 1_2--1_3 
 
 
<!-- 3_4 -->
 
 3_4 
 
 
<!-- 3_5 -->
 
 3_5 
 
 
<!-- 3_4--3_5 -->
 
 3_4--3_5 
 
 
<!-- w0_0 -->
 
 w0_0 
 
 
<!-- w0_1 -->
 
 w0_1 
 
 
<!-- w0_0--w0_1 -->
 
 w0_0--w0_1 
 
 
<!-- w1_0 -->
 
 w1_0 
 
 
<!-- w0_0--w1_0 -->
 
 w0_0--w1_0 
 
 
<!-- 4_9 -->
 
 4_9 
 
 
<!-- w2_0 -->
 
 w2_0 
 
 
<!-- w3_0 -->
 
 w3_0 
 
 
<!-- w2_0--w3_0 -->
 
 w2_0--w3_0 
 
 
<!-- w4_10 -->
 
 w4_10 
 
 
<!-- w3_9 -->
 
 w3_9 
 
 
<!-- 3_8 -->
 
 3_8 
 
 
<!-- 3_9 -->
 
 3_9 
 
 
<!-- 3_8--3_9 -->
 
 3_8--3_9 
 
 
<!-- w3_5 -->
 
 w3_5 
 
 
<!-- w2_5--w3_5 -->
 
 w2_5--w3_5 
 
 
<!-- w1_9 -->
 
 w1_9 
 
 
<!-- w4_1 -->
 
 w4_1 
 
 
<!-- w4_1--w5_1 -->
 
 w4_1--w5_1 
 
 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 0_2--1_2 -->
 
 0_2--1_2 
 
 
<!-- w0_2 -->
 
 w0_2 
 
 
<!-- w0_1--w0_2 -->
 
 w0_1--w0_2 
 
 
<!-- w2_3 -->
 
 w2_3 
 
 
<!-- w2_3--w2_4 -->
 
 w2_3--w2_4 
 
 
<!-- w5_2 -->
 
 w5_2 
 
 
<!-- w5_2--w5_3 -->
 
 w5_2--w5_3 
 
 
<!-- w4_2 -->
 
 w4_2 
 
 
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 2_1 -->
 
 2_1 
 
 
<!-- 2_0--2_1 -->
 
 2_0--2_1 
 
 
<!-- w4_8 -->
 
 w4_8 
 
 
<!-- w4_8--w4_9 -->
 
 w4_8--w4_9 
 
 
<!-- w5_8 -->
 
 w5_8 
 
 
<!-- w4_8--w5_8 -->
 
 w4_8--w5_8 
 
 
<!-- 3_10 -->
 
 3_10 
 
 
<!-- 4_10 -->
 
 4_10 
 
 
<!-- 3_10--4_10 -->
 
 3_10--4_10 
 
 
<!-- w3_1 -->
 
 w3_1 
 
 
<!-- w3_0--w3_1 -->
 
 w3_0--w3_1 
 
 
<!-- w4_0 -->
 
 w4_0 
 
 
<!-- w3_0--w4_0 -->
 
 w3_0--w4_0 
 
 
<!-- w1_7 -->
 
 w1_7 
 
 
<!-- w1_7--w1_8 -->
 
 w1_7--w1_8 
 
 
<!-- 2_10 -->
 
 2_10 
 
 
<!-- 2_10--3_10 -->
 
 2_10--3_10 
 
 
<!-- 2_

Fancier maze presentation with rectangular vertexes and edges (with matching sizes):

In [58]:
#% html
my $g5 = $g4.subgraph($g1.vertex-list);

my @path = $mazePath.find-shortest-path('0_0', "{$n-2}_{$m-2}");
say @path.elems;

my @mazeStartEnd = "w0_0", w0_0 => "w0_1", w0_0 => "w1_0", "w{$n-1}_{$m-1}", "w{$n-1}_{$m-1}" => "w{$n-1}_{$m-2}", "w{$n-1}_{$m-1}" => "w{$n-2}_{$m-1}";

$g4.dot(
    highlight => {'#1F1F1F' => [|$mazePath.vertex-list, |$mazePath.edge-list, |@mazeStartEnd], Orange => @path},
    #highlight => {'#1F1F1F' => @mazeStartEnd},
    background => '#1F1F1F',
    size => '10,10!',
    :!node-labels,
    node-shape => 'square',
    node-color => 'SteelBlue',
    node-fill-color => 'SteelBlue',
    node-width => 0.4,
    edge-width => 30,
    engine => 'neato'
):svg

21


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w5_11 -->
 
 w5_11 
 
 
<!-- 0_6 -->
 
 0_6 
 
 
<!-- 0_7 -->
 
 0_7 
 
 
<!-- 0_6--0_7 -->
 
 0_6--0_7 
 
 
<!-- 1_6 -->
 
 1_6 
 
 
<!-- 0_6--1_6 -->
 
 0_6--1_6 
 
 
<!-- w0_8 -->
 
 w0_8 
 
 
<!-- w0_9 -->
 
 w0_9 
 
 
<!-- w0_8--w0_9 -->
 
 w0_8--w0_9 
 
 
<!-- w1_8 -->
 
 w1_8 
 
 
<!-- w0_8--w1_8 -->
 
 w0_8--w1_8 
 
 
<!-- 4_5 -->
 
 4_5 
 
 
<!-- 0_5 -->
 
 0_5 
 
 
<!-- 4_1 -->
 
 4_1 
 
 
<!-- 4_2 -->
 
 4_2 
 
 
<!-- 4_1--4_2 -->
 
 4_1--4_2 
 
 
<!-- w4_4 -->
 
 w4_4 
 
 
<!-- w4_5 -->
 
 w4_5 
 
 
<!-- w4_4--w4_5 -->
 
 w4_4--w4_5 
 
 
<!-- w5_10 -->
 
 w5_10 
 
 
<!-- w5_9 -->
 
 w5_9 
 
 
<!-- w5_10--w5_9 -->
 
 w5_10--w5_9 
 
 
<!-- w5_3 -->
 
 w5_3 
 
 
<!-- w5_4 -->
 
 w5_4 
 
 
<!-- w5_3--w5_4 -->
 
 w5_3--w5_4 
 
 
<!-- w3_11 -->
 
 w3_11 
 
 
<!-- w4_11 -->
 
 w4_11 
 
 
<!-- w3_11--w4_11 -->
 
 w3_11--w4_11 
 
 
<!-- 0_3 -->
 
 0_3 
 
 
<!-- 0_4 -->
 
 0_4 
 
 
<!-- 0_3--0_4 -->
 
 0_3--0_4 
 
 
<!-- 2_6 -->
 
 2_6 
 
 
<!-- 3_1 -->
 
 3_1 
 
 
<!-- 3_1--4_1 -->
 
 3_1--4_1 
 
 
<!-- w1_5 -->
 
 w1_5 
 
 
<!-- w2_5 -->
 
 w2_5 
 
 
<!-- w1_5--w2_5 -->
 
 w1_5--w2_5 
 
 
<!-- w1_6 -->
 
 w1_6 
 
 
<!-- w1_5--w1_6 -->
 
 w1_5--w1_6 
 
 
<!-- 3_0 -->
 
 3_0 
 
 
<!-- 3_0--3_1 -->
 
 3_0--3_1 
 
 
<!-- 4_0 -->
 
 4_0 
 
 
<!-- 3_0--4_0 -->
 
 3_0--4_0 
 
 
<!-- 3_3 -->
 
 3_3 
 
 
<!-- 1_7 -->
 
 1_7 
 
 
<!-- 2_7 -->
 
 2_7 
 
 
<!-- 1_7--2_7 -->
 
 1_7--2_7 
 
 
<!-- 1_8 -->
 
 1_8 
 
 
<!-- 1_7--1_8 -->
 
 1_7--1_8 
 
 
<!-- w0_4 -->
 
 w0_4 
 
 
<!-- w0_5 -->
 
 w0_5 
 
 
<!-- w0_4--w0_5 -->
 
 w0_4--w0_5 
 
 
<!-- 2_8 -->
 
 2_8 
 
 
<!-- 2_9 -->
 
 2_9 
 
 
<!-- 2_8--2_9 -->
 
 2_8--2_9 
 
 
<!-- 0_4--0_5 -->
 
 0_4--0_5 
 
 
<!-- 1_4 -->
 
 1_4 
 
 
<!-- 0_4--1_4 -->
 
 0_4--1_4 
 
 
<!-- w5_0 -->
 
 w5_0 
 
 
<!-- w5_1 -->
 
 w5_1 
 
 
<!-- w5_0--w5_1 -->
 
 w5_0--w5_1 
 
 
<!-- 3_7 -->
 
 3_7 
 
 
<!-- 4_7 -->
 
 4_7 
 
 
<!-- 3_7--4_7 -->
 
 3_7--4_7 
 
 
<!-- w1_4 -->
 
 w1_4 
 
 
<!-- w2_4 -->
 
 w2_4 
 
 
<!-- w1_4--w2_4 -->
 
 w1_4--w2_4 
 
 
<!-- w4_9 -->
 
 w4_9 
 
 
<!-- 1_2 -->
 
 1_2 
 
 
<!-- 2_2 -->
 
 2_2 
 
 
<!-- 1_2--2_2 -->
 
 1_2--2_2 
 
 
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 1_2--1_3 -->
 
 1_2--1_3 
 
 
<!-- 3_4 -->
 
 3_4 
 
 
<!-- 3_5 -->
 
 3_5 
 
 
<!-- 3_4--3_5 -->
 
 3_4--3_5 
 
 
<!-- w0_0 -->
 
 w0_0 
 
 
<!-- w0_1 -->
 
 w0_1 
 
 
<!-- w0_0--w0_1 -->
 
 w0_0--w0_1 
 
 
<!-- w1_0 -->
 
 w1_0 
 
 
<!-- w0_0--w1_0 -->
 
 w0_0--w1_0 
 
 
<!-- 4_9 -->
 
 4_9 
 
 
<!-- w2_0 -->
 
 w2_0 
 
 
<!-- w3_0 -->
 
 w3_0 
 
 
<!-- w2_0--w3_0 -->
 
 w2_0--w3_0 
 
 
<!-- w4_10 -->
 
 w4_10 
 
 
<!-- w3_9 -->
 
 w3_9 
 
 
<!-- 3_8 -->
 
 3_8 
 
 
<!-- 3_9 -->
 
 3_9 
 
 
<!-- 3_8--3_9 -->
 
 3_8--3_9 
 
 
<!-- w3_5 -->
 
 w3_5 
 
 
<!-- w2_5--w3_5 -->
 
 w2_5--w3_5 
 
 
<!-- w1_9 -->
 
 w1_9 
 
 
<!-- w4_1 -->
 
 w4_1 
 
 
<!-- w4_1--w5_1 -->
 
 w4_1--w5_1 
 
 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 0_2--1_2 -->
 
 0_2--1_2 
 
 
<!-- w0_2 -->
 
 w0_2 
 
 
<!-- w0_1--w0_2 -->
 
 w0_1--w0_2 
 
 
<!-- w2_3 -->
 
 w2_3 
 
 
<!-- w2_3--w2_4 -->
 
 w2_3--w2_4 
 
 
<!-- w5_2 -->
 
 w5_2 
 
 
<!-- w5_2--w5_3 -->
 
 w5_2--w5_3 
 
 
<!-- w4_2 -->
 
 w4_2 
 
 
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 2_1 -->
 
 2_1 
 
 
<!-- 2_0--2_1 -->
 
 2_0--2_1 
 
 
<!-- w4_8 -->
 
 w4_8 
 
 
<!-- w4_8--w4_9 -->
 
 w4_8--w4_9 
 
 
<!-- w5_8 -->
 
 w5_8 
 
 
<!-- w4_8--w5_8 -->
 
 w4_8--w5_8 
 
 
<!-- 3_10 -->
 
 3_10 
 
 
<!-- 4_10 -->
 
 4_10 
 
 
<!-- 3_10--4_10 -->
 
 3_10--4_10 
 
 
<!-- w3_1 -->
 
 w3_1 
 
 
<!-- w3_0--w3_1 -->
 
 w3_0--w3_1 
 
 
<!-- w4_0 -->
 
 w4_0 
 
 
<!-- w3_0--w4_0 -->
 
 w3_0--w4_0 
 
 
<!-- w1_7 -->
 
 w1_7 
 
 
<!-- w1_7--w1_8 -->
 
 w1_7--w1_8 
 
 
<!-- 2_10 -->
 
 2_10 
 
 
<!-- 2_10--3_10 -->
 
 2_10--3_10 
 
 
<!-- 2_10--2_9 -->
 
 2_10--2_9 
 
 
<!-- w4_7 -->

-----

## Hexagonal version

Let us do another maze based on a hexagonal grid. Here are two grid graphs:

- The first is a **hexagonal grid** graph for maze's walls
- The second graph is a **triangular grid** graph with "one less" rows and columns and shifted vertex coordinates

In [86]:
# Maze dimensions
my ($n, $m) = (6, 10) »*» 2;

# Walls graph
my $g1 = Graph::HexagonalGrid.new($n, $m, prefix => 'w');

# Space graph
my $g2 = Graph::TriangularGrid.new($n-1, $m-1);
$g2.vertex-coordinates = $g2.vertex-coordinates.map({ $_.key => $_.value >>+<< [sqrt(3), 1 ] }).Hash;

$g2

Graph(vertexes => 240, edges => 657, directed => False)

In [87]:
#% html
$g1.union($g2).dot(
    highlight => $g2,
    :!node-labels,
    background => "#1F1F1F",
    node-width => 0.85,
    node-height => 0.85,
    node-font-size => 28,
    node-shape => 'hexagon',
    edge-width => 7,
    size => '10,10!',
    engine => 'neato'
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 168 -->
 
 168 
 
 
<!-- 175 -->
 
 175 
 
 
<!-- 168--175 -->
 
 168--175 
 
 
<!-- 180 -->
 
 180 
 
 
<!-- 168--180 -->
 
 168--180 
 
 
<!-- 174 -->
 
 174 
 
 
<!-- 168--174 -->
 
 168--174 
 
 
<!-- w213 -->
 
 w213 
 
 
<!-- w226 -->
 
 w226 
 
 
<!-- w213--w226 -->
 
 w213--w226 
 
 
<!-- 26 -->
 
 26 
 
 
<!-- 37 -->
 
 37 
 
 
<!-- 26--37 -->
 
 26--37 
 
 
<!-- 32 -->
 
 32 
 
 
<!-- 26--32 -->
 
 26--32 
 
 
<!-- 31 -->
 
 31 
 
 
<!-- 26--31 -->
 
 26--31 
 
 
<!-- w391 -->
 
 w391 
 
 
<!-- w404 -->
 
 w404 
 
 
<!-- w391--w404 -->
 
 w391--w404 
 
 
<!-- w237 -->
 
 w237 
 
 
<!-- w250 -->
 
 w250 
 
 
<!-- w237--w250 -->
 
 w237--w250 
 
 
<!-- 35 -->
 
 35 
 
 
<!-- 41 -->
 
 41 
 
 
<!-- 35--41 -->
 
 35--41 
 
 
<!-- 47 -->
 
 47 
 
 
<!-- 35--47 -->
 
 35--47 
 
 
<!-- 40 -->
 
 40 
 
 
<!-- 35--40 -->
 
 35--40 
 
 
<!-- 78 -->
 
 78 
 
 
<!-- 90 -->
 
 90 
 
 
<!-- 78--90 -->
 
 78--90 
 
 
<!-- 84 -->
 
 84 
 
 
<!-- 78--84 -->
 
 78--84 
 
 
<!-- w454 -->
 
 w454 
 
 
<!-- w455 -->
 
 w455 
 
 
<!-- w454--w455 -->
 
 w454--w455 
 
 
<!-- w162 -->
 
 w162 
 
 
<!-- w163 -->
 
 w163 
 
 
<!-- w162--w163 -->
 
 w162--w163 
 
 
<!-- w175 -->
 
 w175 
 
 
<!-- w162--w175 -->
 
 w162--w175 
 
 
<!-- w282 -->
 
 w282 
 
 
<!-- w283 -->
 
 w283 
 
 
<!-- w282--w283 -->
 
 w282--w283 
 
 
<!-- w295 -->
 
 w295 
 
 
<!-- w282--w295 -->
 
 w282--w295 
 
 
<!-- w430 -->
 
 w430 
 
 
<!-- w431 -->
 
 w431 
 
 
<!-- w430--w431 -->
 
 w430--w431 
 
 
<!-- w443 -->
 
 w443 
 
 
<!-- w430--w443 -->
 
 w430--w443 
 
 
<!-- w227 -->
 
 w227 
 
 
<!-- w226--w227 -->
 
 w226--w227 
 
 
<!-- w239 -->
 
 w239 
 
 
<!-- w226--w239 -->
 
 w226--w239 
 
 
<!-- w218 -->
 
 w218 
 
 
<!-- w219 -->
 
 w219 
 
 
<!-- w218--w219 -->
 
 w218--w219 
 
 
<!-- w231 -->
 
 w231 
 
 
<!-- w218--w231 -->
 
 w218--w231 
 
 
<!-- w70 -->
 
 w70 
 
 
<!-- w82 -->
 
 w82 
 
 
<!-- w70--w82 -->
 
 w70--w82 
 
 
<!-- w105 -->
 
 w105 
 
 
<!-- w92 -->
 
 w92 
 
 
<!-- w105--w92 -->
 
 w105--w92 
 
 
<!-- w118 -->
 
 w118 
 
 
<!-- w105--w118 -->
 
 w105--w118 
 
 
<!-- w318 -->
 
 w318 
 
 
<!-- w319 -->
 
 w319 
 
 
<!-- w318--w319 -->
 
 w318--w319 
 
 
<!-- w331 -->
 
 w331 
 
 
<!-- w318--w331 -->
 
 w318--w331 
 
 
<!-- w192 -->
 
 w192 
 
 
<!-- w193 -->
 
 w193 
 
 
<!-- w192--w193 -->
 
 w192--w193 
 
 
<!-- w205 -->
 
 w205 
 
 
<!-- w192--w205 -->
 
 w192--w205 
 
 
<!-- w236 -->
 
 w236 
 
 
<!-- w236--w237 -->
 
 w236--w237 
 
 
<!-- w249 -->
 
 w249 
 
 
<!-- w236--w249 -->
 
 w236--w249 
 
 
<!-- w165 -->
 
 w165 
 
 
<!-- w178 -->
 
 w178 
 
 
<!-- w165--w178 -->
 
 w165--w178 
 
 
<!-- 204 -->
 
 204 
 
 
<!-- 210 -->
 
 210 
 
 
<!-- 204--210 -->
 
 204--210 
 
 
<!-- w62 -->
 
 w62 
 
 
<!-- w73 -->
 
 w73 
 
 
<!-- w62--w73 -->
 
 w62--w73 
 
 
<!-- w322 -->
 
 w322 
 
 
<!-- w323 -->
 
 w323 
 
 
<!-- w322--w323 -->
 
 w322--w323 
 
 
<!-- w335 -->
 
 w335 
 
 
<!-- w322--w335 -->
 
 w322--w335 
 
 
<!-- w305 -->
 
 w305 
 
 
<!-- w305--w318 -->
 
 w305--w318 
 
 
<!-- w317 -->
 
 w317 
 
 
<!-- w330 -->
 
 w330 
 
 
<!-- w317--w330 -->
 
 w317--w330 
 
 
<!-- w361 -->
 
 w361 
 
 
<!-- w374 -->
 
 w374 
 
 
<!-- w361--w374 -->
 
 w361--w374 
 
 
<!-- w215 -->
 
 w215 
 
 
<!-- w228 -->
 
 w228 
 
 
<!-- w215--w228 -->
 
 w215--w228 
 
 
<!-- w96 -->
 
 w96 
 
 
<!-- w97 -->
 
 w97 
 
 
<!-- w96--w97 -->
 
 w96--w97 
 
 
<!-- 67 -->
 
 67 
 
 
<!-- 79 -->
 
 79 
 
 
<!-- 67--79 -->
 
 67--79 
 
 
<!-- 72 -->
 
 72 
 
 
<!-- 67--72 -->
 
 67--72 
 
 
<!-- 73 -->
 
 73 
 
 
<!-- 67--73 -->
 
 67--73 
 
 
<!-- 187 -->
 
 187 
 
 
<!-- 192 -->
 
 192 
 
 
<!-- 187--192 -->
 
 187--192 
 
 
<!-- 199 -->
 
 199 
 
 
<!-- 187--199 -->
 
 187--199 
 
 
<!-- 193 -->
 
 193 
 
 
<!-- 187--193 -

Maze path graph:

In [88]:
my $mazePath = Graph.new($g2.edges(:dataset).map({ $_<weight> = random-real([10,1000]); $_}));
$mazePath = $mazePath.find-spanning-tree;
$mazePath = Graph.new($mazePath.edges);
$mazePath.vertex-coordinates = $g2.vertex-coordinates;

$mazePath

Graph(vertexes => 240, edges => 239, directed => False)

Combine the walls-maze and the maze-path graphs (i.e. make a union of them) and plot the result graph:

In [89]:
#% html

my $g3 = $g1.union($mazePath);

$g3.dot(
    highlight => $mazePath,
    :node-labels,
    background => "#1F1F1F",
    node-width => 0.85,
    node-height => 0.85,
    node-font-size => 28,
    node-shape => 'hexagon',
    edge-width => 7,
    size => '10,10!',
    engine => 'neato'
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w23 -->
 
 w23 
 
 w23 
 
<!-- w30 -->
 
 w30 
 
 w30 
 
<!-- w23--w30 -->
 
 w23--w30 
 
 
<!-- w24 -->
 
 w24 
 
 w24 
 
<!-- w23--w24 -->
 
 w23--w24 
 
 
<!-- w201 -->
 
 w201 
 
 w201 
 
<!-- w214 -->
 
 w214 
 
 w214 
 
<!-- w201--w214 -->
 
 w201--w214 
 
 
<!-- w264 -->
 
 w264 
 
 w264 
 
<!-- w277 -->
 
 w277 
 
 w277 
 
<!-- w264--w277 -->
 
 w264--w277 
 
 
<!-- w265 -->
 
 w265 
 
 w265 
 
<!-- w264--w265 -->
 
 w264--w265 
 
 
<!-- w344 -->
 
 w344 
 
 w344 
 
<!-- w345 -->
 
 w345 
 
 w345 
 
<!-- w344--w345 -->
 
 w344--w345 
 
 
<!-- w357 -->
 
 w357 
 
 w357 
 
<!-- w344--w357 -->
 
 w344--w357 
 
 
<!-- w44 -->
 
 w44 
 
 w44 
 
<!-- w45 -->
 
 w45 
 
 w45 
 
<!-- w44--w45 -->
 
 w44--w45 
 
 
<!-- w54 -->
 
 w54 
 
 w54 
 
<!-- w44--w54 -->
 
 w44--w54 
 
 
<!-- w40 -->
 
 w40 
 
 w40 
 
<!-- w41 -->
 
 w41 
 
 w41 
 
<!-- w40--w41 -->
 
 w40--w41 
 
 
<!-- w49 -->
 
 w49 
 
 w49 
 
<!-- w40--w49 -->
 
 w40--w49 
 
 
<!-- w338 -->
 
 w338 
 
 w338 
 
<!-- w339 -->
 
 w339 
 
 w339 
 
<!-- w338--w339 -->
 
 w338--w339 
 
 
<!-- w351 -->
 
 w351 
 
 w351 
 
<!-- w338--w351 -->
 
 w338--w351 
 
 
<!-- w449 -->
 
 w449 
 
 w449 
 
<!-- w462 -->
 
 w462 
 
 w462 
 
<!-- w449--w462 -->
 
 w449--w462 
 
 
<!-- 166 -->
 
 166 
 
 166 
 
<!-- w532 -->
 
 w532 
 
 w532 
 
<!-- w536 -->
 
 w536 
 
 w536 
 
<!-- w532--w536 -->
 
 w532--w536 
 
 
<!-- w533 -->
 
 w533 
 
 w533 
 
<!-- w532--w533 -->
 
 w532--w533 
 
 
<!-- w31 -->
 
 w31 
 
 w31 
 
<!-- w32 -->
 
 w32 
 
 w32 
 
<!-- w31--w32 -->
 
 w31--w32 
 
 
<!-- w39 -->
 
 w39 
 
 w39 
 
<!-- w31--w39 -->
 
 w31--w39 
 
 
<!-- w399 -->
 
 w399 
 
 w399 
 
<!-- w412 -->
 
 w412 
 
 w412 
 
<!-- w399--w412 -->
 
 w399--w412 
 
 
<!-- w4 -->
 
 w4 
 
 w4 
 
<!-- w7 -->
 
 w7 
 
 w7 
 
<!-- w4--w7 -->
 
 w4--w7 
 
 
<!-- w215 -->
 
 w215 
 
 w215 
 
<!-- w214--w215 -->
 
 w214--w215 
 
 
<!-- w227 -->
 
 w227 
 
 w227 
 
<!-- w214--w227 -->
 
 w214--w227 
 
 
<!-- w475 -->
 
 w475 
 
 w475 
 
<!-- w476 -->
 
 w476 
 
 w476 
 
<!-- w475--w476 -->
 
 w475--w476 
 
 
<!-- w486 -->
 
 w486 
 
 w486 
 
<!-- w475--w486 -->
 
 w475--w486 
 
 
<!-- w180 -->
 
 w180 
 
 w180 
 
<!-- w193 -->
 
 w193 
 
 w193 
 
<!-- w180--w193 -->
 
 w180--w193 
 
 
<!-- 24 -->
 
 24 
 
 24 
 
<!-- 34 -->
 
 34 
 
 34 
 
<!-- 24--34 -->
 
 24--34 
 
 
<!-- w82 -->
 
 w82 
 
 w82 
 
<!-- w95 -->
 
 w95 
 
 w95 
 
<!-- w82--w95 -->
 
 w82--w95 
 
 
<!-- w83 -->
 
 w83 
 
 w83 
 
<!-- w82--w83 -->
 
 w82--w83 
 
 
<!-- w252 -->
 
 w252 
 
 w252 
 
<!-- w252--w265 -->
 
 w252--w265 
 
 
<!-- w253 -->
 
 w253 
 
 w253 
 
<!-- w252--w253 -->
 
 w252--w253 
 
 
<!-- w358 -->
 
 w358 
 
 w358 
 
<!-- w345--w358 -->
 
 w345--w358 
 
 
<!-- w422 -->
 
 w422 
 
 w422 
 
<!-- w435 -->
 
 w435 
 
 w435 
 
<!-- w422--w435 -->
 
 w422--w435 
 
 
<!-- w423 -->
 
 w423 
 
 w423 
 
<!-- w422--w423 -->
 
 w422--w423 
 
 
<!-- w430 -->
 
 w430 
 
 w430 
 
<!-- w443 -->
 
 w443 
 
 w443 
 
<!-- w430--w443 -->
 
 w430--w443 
 
 
<!-- w431 -->
 
 w431 
 
 w431 
 
<!-- w430--w431 -->
 
 w430--w431 
 
 
<!-- w76 -->
 
 w76 
 
 w76 
 
<!-- w88 -->
 
 w88 
 
 w88 
 
<!-- w76--w88 -->
 
 w76--w88 
 
 
<!-- w34 -->
 
 w34 
 
 w34 
 
<!-- w42 -->
 
 w42 
 
 w42 
 
<!-- w34--w42 -->
 
 w34--w42 
 
 
<!-- 71 -->
 
 71 
 
 71 
 
<!-- 77 -->
 
 77 
 
 77 
 
<!-- 71--77 -->
 
 71--77 
 
 
<!-- 231 -->
 
 231 
 
 231 
 
<!-- 234 -->
 
 234 
 
 234 
 
<!-- 231--234 -->
 
 231--234 
 
 
<!-- w79 -->
 
 w79 
 
 w79 
 
<!-- w92 -->
 
 w92 
 
 w92 
 
<!-- w79--w92 -->
 
 w79--w92 
 
 
<!-- w432 -->
 
 w432 
 
 w432 
 
<!-- w433 -->
 
 w433 
 
 w433 
 
<!-- w432--w433 -->
 
 w432--w433 
 
 
<!-- w445 -->
 
 w445 
 
 w445 
 
<!-- w432--w445 -->
 
 w432--w445 
 
 
<!-- w522 -->
 
 w522 
 
 w522

Make a nearest neighbor points finder functor:

In [90]:
my &finder = nearest($g1.vertex-coordinates.Array, method => 'KDTree');


Math::Nearest::Finder(Algorithm::KDimensionalTree(points => 544, distance-function => &euclidean-distance, labels => 544))

Take a maze edge and its vertex points:

In [91]:
my $e = $mazePath.edges.head;
my @points = $g2.vertex-coordinates{($e.kv)};

[[-3.4641016151381225 -2] [-1.2279066652354231e-13 -2]]

Find the edge's mean point and the nearest wall-graph vertexes:

In [92]:
my @m = |((@points.head <<+>> @points.tail) >>/>> 2);
say "Mean edge point: {@m}";
my @vs = |&finder.nearest(@m, 2, prop => <label>).flat;
say "To remove: {@vs}";

Mean edge point: -1.7320508075691228 -2
To remove: w5 w6


Loop over all maze edges with removal of wall-maze edges:

In [93]:
my $g4 = $g1.clone;
for $mazePath.edges -> $e {
    my @points = $g2.vertex-coordinates{($e.kv)};
    my @m = |((@points.head <<+>> @points.tail) >>/>> 2);
    my @vs = |&finder.nearest(@m, 2, prop => <label>).flat;
    $g4 = $g4.edge-delete(@vs.head => @vs.tail);
}

$g4

Graph(vertexes => 544, edges => 544, directed => False)

The start and end points of the maze:

In [94]:
my ($start, $end) = $g4.vertex-list.head, $g4.vertex-list.sort({ $_.substr(1).Int }).tail;

(w0 w543)

Finding the maze solution:

In [96]:
my $solution = Graph::Path.new: $mazePath.find-shortest-path(|$mazePath.vertex-list.sort(*.Int)[0,*-1]);
$solution.vertex-coordinates = $mazePath.vertex-coordinates.grep({$_.key ∈ $solution.vertex-list }).Hash;

$solution

Graph(vertexes => 62, edges => 61, directed => False)

Plot the maze:

In [97]:
#% html

my @mazeStartEnd = $start, $end, |$g4.neighborhood-graph([$start, $end]).edges;

my $g5 = $g4.union($solution);

my %opts = 
    :!node-labels,
    background => "#1F1F1F",
    node-width => 0.8,
    node-height => 0.8,
    node-shape => 'circle',
    edge-width => 40,
    size => '10,10!',
    engine => 'neato';

$g4.dot(highlight => {'#1F1F1F' => @mazeStartEnd}, |%opts):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w163 -->
 
 w163 
 
 
<!-- w176 -->
 
 w176 
 
 
<!-- w163--w176 -->
 
 w163--w176 
 
 
<!-- w18 -->
 
 w18 
 
 
<!-- w19 -->
 
 w19 
 
 
<!-- w18--w19 -->
 
 w18--w19 
 
 
<!-- w24 -->
 
 w24 
 
 
<!-- w18--w24 -->
 
 w18--w24 
 
 
<!-- w14 -->
 
 w14 
 
 
<!-- w20 -->
 
 w20 
 
 
<!-- w14--w20 -->
 
 w14--w20 
 
 
<!-- w9 -->
 
 w9 
 
 
<!-- w14--w9 -->
 
 w14--w9 
 
 
<!-- w76 -->
 
 w76 
 
 
<!-- w88 -->
 
 w88 
 
 
<!-- w76--w88 -->
 
 w76--w88 
 
 
<!-- w68 -->
 
 w68 
 
 
<!-- w80 -->
 
 w80 
 
 
<!-- w68--w80 -->
 
 w68--w80 
 
 
<!-- w105 -->
 
 w105 
 
 
<!-- w92 -->
 
 w92 
 
 
<!-- w105--w92 -->
 
 w105--w92 
 
 
<!-- w118 -->
 
 w118 
 
 
<!-- w105--w118 -->
 
 w105--w118 
 
 
<!-- w38 -->
 
 w38 
 
 
<!-- w47 -->
 
 w47 
 
 
<!-- w38--w47 -->
 
 w38--w47 
 
 
<!-- w468 -->
 
 w468 
 
 
<!-- w479 -->
 
 w479 
 
 
<!-- w468--w479 -->
 
 w468--w479 
 
 
<!-- w368 -->
 
 w368 
 
 
<!-- w381 -->
 
 w381 
 
 
<!-- w368--w381 -->
 
 w368--w381 
 
 
<!-- w71 -->
 
 w71 
 
 
<!-- w83 -->
 
 w83 
 
 
<!-- w71--w83 -->
 
 w71--w83 
 
 
<!-- w441 -->
 
 w441 
 
 
<!-- w454 -->
 
 w454 
 
 
<!-- w441--w454 -->
 
 w441--w454 
 
 
<!-- w440 -->
 
 w440 
 
 
<!-- w453 -->
 
 w453 
 
 
<!-- w440--w453 -->
 
 w440--w453 
 
 
<!-- w37 -->
 
 w37 
 
 
<!-- w46 -->
 
 w46 
 
 
<!-- w37--w46 -->
 
 w37--w46 
 
 
<!-- w220 -->
 
 w220 
 
 
<!-- w233 -->
 
 w233 
 
 
<!-- w220--w233 -->
 
 w220--w233 
 
 
<!-- w255 -->
 
 w255 
 
 
<!-- w480 -->
 
 w480 
 
 
<!-- w490 -->
 
 w490 
 
 
<!-- w480--w490 -->
 
 w480--w490 
 
 
<!-- w429 -->
 
 w429 
 
 
<!-- w203 -->
 
 w203 
 
 
<!-- w216 -->
 
 w216 
 
 
<!-- w203--w216 -->
 
 w203--w216 
 
 
<!-- w336 -->
 
 w336 
 
 
<!-- w349 -->
 
 w349 
 
 
<!-- w336--w349 -->
 
 w336--w349 
 
 
<!-- w141 -->
 
 w141 
 
 
<!-- w154 -->
 
 w154 
 
 
<!-- w141--w154 -->
 
 w141--w154 
 
 
<!-- w505 -->
 
 w505 
 
 
<!-- w345 -->
 
 w345 
 
 
<!-- w358 -->
 
 w358 
 
 
<!-- w345--w358 -->
 
 w345--w358 
 
 
<!-- w265 -->
 
 w265 
 
 
<!-- w521 -->
 
 w521 
 
 
<!-- w522 -->
 
 w522 
 
 
<!-- w521--w522 -->
 
 w521--w522 
 
 
<!-- w527 -->
 
 w527 
 
 
<!-- w521--w527 -->
 
 w521--w527 
 
 
<!-- w339 -->
 
 w339 
 
 
<!-- w27 -->
 
 w27 
 
 
<!-- w20--w27 -->
 
 w20--w27 
 
 
<!-- w298 -->
 
 w298 
 
 
<!-- w311 -->
 
 w311 
 
 
<!-- w298--w311 -->
 
 w298--w311 
 
 
<!-- w506 -->
 
 w506 
 
 
<!-- w514 -->
 
 w514 
 
 
<!-- w506--w514 -->
 
 w506--w514 
 
 
<!-- w507 -->
 
 w507 
 
 
<!-- w506--w507 -->
 
 w506--w507 
 
 
<!-- w459 -->
 
 w459 
 
 
<!-- w122 -->
 
 w122 
 
 
<!-- w135 -->
 
 w135 
 
 
<!-- w122--w135 -->
 
 w122--w135 
 
 
<!-- w501 -->
 
 w501 
 
 
<!-- w509 -->
 
 w509 
 
 
<!-- w501--w509 -->
 
 w501--w509 
 
 
<!-- w520 -->
 
 w520 
 
 
<!-- w234 -->
 
 w234 
 
 
<!-- w235 -->
 
 w235 
 
 
<!-- w234--w235 -->
 
 w234--w235 
 
 
<!-- w247 -->
 
 w247 
 
 
<!-- w234--w247 -->
 
 w234--w247 
 
 
<!-- w200 -->
 
 w200 
 
 
<!-- w201 -->
 
 w201 
 
 
<!-- w200--w201 -->
 
 w200--w201 
 
 
<!-- w178 -->
 
 w178 
 
 
<!-- w191 -->
 
 w191 
 
 
<!-- w178--w191 -->
 
 w178--w191 
 
 
<!-- w179 -->
 
 w179 
 
 
<!-- w178--w179 -->
 
 w178--w179 
 
 
<!-- w169 -->
 
 w169 
 
 
<!-- w257 -->
 
 w257 
 
 
<!-- w210 -->
 
 w210 
 
 
<!-- w211 -->
 
 w211 
 
 
<!-- w210--w211 -->
 
 w210--w211 
 
 
<!-- w223 -->
 
 w223 
 
 
<!-- w210--w223 -->
 
 w210--w223 
 
 
<!-- w252 -->
 
 w252 
 
 
<!-- w252--w265 -->
 
 w252--w265 
 
 
<!-- w219 -->
 
 w219 
 
 
<!-- w232 -->
 
 w232 
 
 
<!-- w219--w232 -->
 
 w219--w232 
 
 
<!-- w195 -->
 
 w195 
 
 
<!-- w208 -->
 
 w208 
 
 
<!-- w195--w208 -->
 
 w195--w208 
 
 
<!-- w330 -->
 
 w330 
 
 
<!-- w343 -->
 
 w343 
 
 
<!-- w330--w343 -->
 
 w330--w343 
 
 
<!-- w331 -->
 
 w331 
 
 
<!-- w330--w331 -->
 
 w330

In [98]:
#% html
my %highlight = '#1F1F1F' => @mazeStartEnd, DarkOrange => [|$solution.vertex-list, |$solution.edges];
my $res = $g5.dot(:%highlight, |%opts);

#highlight => {'#1F1F1F' => @mazeStartEnd},

$res .= subst('[fillcolor="DarkOrange", color="DarkOrange"]', '[fillcolor="DarkOrange", color="DarkOrange", shape = circle, height = 0.5, width = 0.5]', :g);
$res .= subst('edge [color="DarkOrange", penwidth=40, arrowsize=1]', 'edge [color="Salmon", penwidth=5, arrowsize=1]', :g);
dot-svg($res, engine => 'neato')

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- w74 -->
 
 w74 
 
 
<!-- w86 -->
 
 w86 
 
 
<!-- w74--w86 -->
 
 w74--w86 
 
 
<!-- w467 -->
 
 w467 
 
 
<!-- w468 -->
 
 w468 
 
 
<!-- w467--w468 -->
 
 w467--w468 
 
 
<!-- w354 -->
 
 w354 
 
 
<!-- w367 -->
 
 w367 
 
 
<!-- w354--w367 -->
 
 w354--w367 
 
 
<!-- w238 -->
 
 w238 
 
 
<!-- w239 -->
 
 w239 
 
 
<!-- w238--w239 -->
 
 w238--w239 
 
 
<!-- w441 -->
 
 w441 
 
 
<!-- w454 -->
 
 w454 
 
 
<!-- w441--w454 -->
 
 w441--w454 
 
 
<!-- w519 -->
 
 w519 
 
 
<!-- w520 -->
 
 w520 
 
 
<!-- w519--w520 -->
 
 w519--w520 
 
 
<!-- w375 -->
 
 w375 
 
 
<!-- w388 -->
 
 w388 
 
 
<!-- w375--w388 -->
 
 w375--w388 
 
 
<!-- w260 -->
 
 w260 
 
 
<!-- w416 -->
 
 w416 
 
 
<!-- w331 -->
 
 w331 
 
 
<!-- 98 -->
 
 98 
 
 
<!-- w137 -->
 
 w137 
 
 
<!-- w150 -->
 
 w150 
 
 
<!-- w137--w150 -->
 
 w137--w150 
 
 
<!-- w21 -->
 
 w21 
 
 
<!-- w28 -->
 
 w28 
 
 
<!-- w21--w28 -->
 
 w21--w28 
 
 
<!-- w22 -->
 
 w22 
 
 
<!-- w21--w22 -->
 
 w21--w22 
 
 
<!-- w149 -->
 
 w149 
 
 
<!-- w162 -->
 
 w162 
 
 
<!-- w149--w162 -->
 
 w149--w162 
 
 
<!-- w129 -->
 
 w129 
 
 
<!-- w142 -->
 
 w142 
 
 
<!-- w129--w142 -->
 
 w129--w142 
 
 
<!-- w352 -->
 
 w352 
 
 
<!-- w365 -->
 
 w365 
 
 
<!-- w352--w365 -->
 
 w352--w365 
 
 
<!-- w353 -->
 
 w353 
 
 
<!-- w352--w353 -->
 
 w352--w353 
 
 
<!-- 133 -->
 
 133 
 
 
<!-- 145 -->
 
 145 
 
 
<!-- 133--145 -->
 
 133--145 
 
 
<!-- w43 -->
 
 w43 
 
 
<!-- w52 -->
 
 w52 
 
 
<!-- w43--w52 -->
 
 w43--w52 
 
 
<!-- w327 -->
 
 w327 
 
 
<!-- w340 -->
 
 w340 
 
 
<!-- w327--w340 -->
 
 w327--w340 
 
 
<!-- w528 -->
 
 w528 
 
 
<!-- w529 -->
 
 w529 
 
 
<!-- w528--w529 -->
 
 w528--w529 
 
 
<!-- w533 -->
 
 w533 
 
 
<!-- w528--w533 -->
 
 w528--w533 
 
 
<!-- w5 -->
 
 w5 
 
 
<!-- w9 -->
 
 w9 
 
 
<!-- w5--w9 -->
 
 w5--w9 
 
 
<!-- w44 -->
 
 w44 
 
 
<!-- w45 -->
 
 w45 
 
 
<!-- w44--w45 -->
 
 w44--w45 
 
 
<!-- w54 -->
 
 w54 
 
 
<!-- w44--w54 -->
 
 w44--w54 
 
 
<!-- w219 -->
 
 w219 
 
 
<!-- w232 -->
 
 w232 
 
 
<!-- w219--w232 -->
 
 w219--w232 
 
 
<!-- w67 -->
 
 w67 
 
 
<!-- w79 -->
 
 w79 
 
 
<!-- w67--w79 -->
 
 w67--w79 
 
 
<!-- w68 -->
 
 w68 
 
 
<!-- w67--w68 -->
 
 w67--w68 
 
 
<!-- w14 -->
 
 w14 
 
 
<!-- w14--w9 -->
 
 w14--w9 
 
 
<!-- w20 -->
 
 w20 
 
 
<!-- w14--w20 -->
 
 w14--w20 
 
 
<!-- w450 -->
 
 w450 
 
 
<!-- w463 -->
 
 w463 
 
 
<!-- w450--w463 -->
 
 w450--w463 
 
 
<!-- 34 -->
 
 34 
 
 
<!-- 46 -->
 
 46 
 
 
<!-- 34--46 -->
 
 34--46 
 
 
<!-- w302 -->
 
 w302 
 
 
<!-- w315 -->
 
 w315 
 
 
<!-- w302--w315 -->
 
 w302--w315 
 
 
<!-- w303 -->
 
 w303 
 
 
<!-- w302--w303 -->
 
 w302--w303 
 
 
<!-- w320 -->
 
 w320 
 
 
<!-- w321 -->
 
 w321 
 
 
<!-- w320--w321 -->
 
 w320--w321 
 
 
<!-- w316 -->
 
 w316 
 
 
<!-- w317 -->
 
 w317 
 
 
<!-- w316--w317 -->
 
 w316--w317 
 
 
<!-- w329 -->
 
 w329 
 
 
<!-- w316--w329 -->
 
 w316--w329 
 
 
<!-- w336 -->
 
 w336 
 
 
<!-- w349 -->
 
 w349 
 
 
<!-- w336--w349 -->
 
 w336--w349 
 
 
<!-- w216 -->
 
 w216 
 
 
<!-- w229 -->
 
 w229 
 
 
<!-- w216--w229 -->
 
 w216--w229 
 
 
<!-- w217 -->
 
 w217 
 
 
<!-- w216--w217 -->
 
 w216--w217 
 
 
<!-- w279 -->
 
 w279 
 
 
<!-- w292 -->
 
 w292 
 
 
<!-- w279--w292 -->
 
 w279--w292 
 
 
<!-- w412 -->
 
 w412 
 
 
<!-- w413 -->
 
 w413 
 
 
<!-- w412--w413 -->
 
 w412--w413 
 
 
<!-- w425 -->
 
 w425 
 
 
<!-- w412--w425 -->
 
 w412--w425 
 
 
<!-- w409 -->
 
 w409 
 
 
<!-- w417 -->
 
 w417 
 
 
<!-- w430 -->
 
 w430 
 
 
<!-- w417--w430 -->
 
 w417--w430 
 
 
<!-- w341 -->
 
 w341 
 
 
<!-- w381 -->
 
 w381 
 
 
<!-- w241 -->
 
 w241 
 
 
<!-- w81 -->
 
 w81 
 
 
<!-- w94 -->
 
 w94 
 
 
<!-- w81--w94 -->
 
 w81--w94 
 
 
<!-- w457 -->
 
 w457 
 
 
<!-- w469 -->
 
 w469 
 
 
<!-

---

## Additional comments


- The initial (and unfinished) version of this document was made 13 months ago.

    - Its finishing was postponed, since the blog post ["Day 12 – Graphs in Raku"](https://raku-advent.blog/2024/12/12/day-12-graphs-in-raku/), [AA1], featured many of the graph operations in "Graph", [AAp1]. 

        - (Well, until this Raku advent effort...)

- The document demonstrates how feature rich the package "Graph" is.

- Here are the special graph functionalities used to make the mazes:

    - Construction of [regular grid graph](https://mathworld.wolfram.com/GridGraph.html)
    - Construction of [hexagonal grid graph](https://mathworld.wolfram.com/HexagonalGridGraph.html)
    - Construction of [triangular grid graph](https://mathworld.wolfram.com/TriangularGridGraph.html)
    - [Subgraph](https://mathworld.wolfram.com/Subgraph.html) taking
    - [Neighborhood graphs](https://mathworld.wolfram.com/NeighborhoodGraph.html)
    - [Graph difference](https://mathworld.wolfram.com/GraphDifference.html)
    - Edge deletion
    - Graph plotting via Graphviz DOT using:
        - Customized styling of various elements
        - Vertex coordinates
        - Specified vertex labels (see the top of the tree)
    - Graph highlighting
        - Multiple sets of vertices and edges with different colors can be specified


----

## References

### Articles, blog posts

[AA1] Anton Antonov,
[“Day 12 – Graphs in Raku”](https://raku-advent.blog/2024/12/12/day-12-graphs-in-raku/),
(2024),
[Raku Advent Calendar at WordPress](https://raku-advent.blog/).

### Packages

[AAp1] Anton Antonov,
[Graph, Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[Math::Nearest, Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

### Videos

[AAv1] Anton Antonov,